In [62]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, col
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType
from time import sleep

In [107]:
sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Lab7_1")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

In [108]:
dataSchema = StructType(
    [StructField("Tweet", StringType(), True),
     StructField("Sentiment", StringType(), True),
     StructField("Party", StringType(), True)
     ])

# Read from a source 
sdf = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1).option("delimiter", ";") \
    .csv("/home/jovyan/data/tweets")

In [109]:
# this works:
tweet_counts = sdf.groupBy("Party", "Sentiment").count()
party_query = tweet_counts.writeStream.queryName("party_counts").outputMode("complete").format("memory").start()
for x in range(3):
    spark.sql("SELECT * FROM party_counts").show()
    sleep(15)

'\nthis works:\ntweet_counts = sdf.groupBy("Party", "Sentiment").count()\nparty_query = tweet_counts.writeStream.queryName("party_counts").outputMode("complete").format("memory").start()\nfor x in range(3):\n    spark.sql("SELECT * FROM party_counts").show()\n    sleep(15)\n'

In [110]:
# this does not work
positive_df = sdf.where(sdf.Sentiment == "positive")
positive_counts = positive_df.groupBy("Party").count()

all_counts = sdf.groupBy("Party").count()

df = positive_counts.join(all_counts)

party_query = df.writeStream.queryName("bla").outputMode("complete").format("memory").start()
for x in range(3):
    spark.sql("SELECT * FROM bla").show()
    sleep(15)

AnalysisException: Multiple streaming aggregations are not supported with streaming DataFrames/Datasets;
Join Inner
:- Aggregate [Party#1449], [Party#1449, count(1) AS count#1457L]
:  +- Filter (Sentiment#1448 = positive)
:     +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@5a250bef,csv,List(),Some(StructType(StructField(Tweet,StringType,true), StructField(Sentiment,StringType,true), StructField(Party,StringType,true))),List(),None,Map(maxFilesPerTrigger -> 1, delimiter -> ;, path -> /home/jovyan/data/tweets),None), FileSource[/home/jovyan/data/tweets], [Tweet#1447, Sentiment#1448, Party#1449]
+- Aggregate [Party#1469], [Party#1469, count(1) AS count#1464L]
   +- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@5a250bef,csv,List(),Some(StructType(StructField(Tweet,StringType,true), StructField(Sentiment,StringType,true), StructField(Party,StringType,true))),List(),None,Map(maxFilesPerTrigger -> 1, delimiter -> ;, path -> /home/jovyan/data/tweets),None),

In [106]:
# Stop the spark context
spark.stop()